In [1]:
import pandas as pd
import numpy as np
import re
import subprocess
from pathlib import Path 
import io
import dvc.api, dvc.repo

In [24]:
url = "https://github.com/mostafa-fallaha/data-drift-simulation"
data = dvc.api.read("new_data/Google-Playstore.csv", encoding='utf-8', repo=url)
df = pd.read_csv(io.StringIO(data))
df.head()

,Category,Rating,Rating Count,Installs,Free,Price,Size_M,Content Rating,Ad Supported,In App Purchases,Editors Choice,Released_Year,Released_Month,Days_Between,Daily_Avg_Installs
0,Adventure,0.0,0.0,15,True,0.0,10.0,Everyone,False,False,False,2020,2,0,15.0
1,Productivity,0.0,0.0,58,True,0.0,3.7,Everyone,False,False,False,2019,8,10,5.8
2,Libraries & Demo,4.5,12.0,2567,True,0.0,2.5,Everyone,True,False,False,2019,9,4,641.8
3,Lifestyle,2.0,39.0,702,True,0.0,16.0,Everyone,False,False,False,2019,6,0,702.0
4,Racing,4.9,55.0,329,True,0.0,51.0,Everyone,False,False,False,2020,7,0,329.0


In [25]:
df.shape

(90000, 15)

In [26]:
df.columns

Index(['Category', 'Rating', 'Rating Count', 'Installs', 'Free', 'Price',
       'Size_M', 'Content Rating', 'Ad Supported', 'In App Purchases',
       'Editors Choice', 'Released_Year', 'Released_Month', 'Days_Between',
       'Daily_Avg_Installs'],
      dtype='object')

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90000 entries, 0 to 89999
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Category            90000 non-null  object 
 1   Rating              90000 non-null  float64
 2   Rating Count        90000 non-null  float64
 3   Installs            90000 non-null  int64  
 4   Free                90000 non-null  bool   
 5   Price               90000 non-null  float64
 6   Size_M              90000 non-null  float64
 7   Content Rating      90000 non-null  object 
 8   Ad Supported        90000 non-null  bool   
 9   In App Purchases    90000 non-null  bool   
 10  Editors Choice      90000 non-null  bool   
 11  Released_Year       90000 non-null  int64  
 12  Released_Month      90000 non-null  int64  
 13  Days_Between        90000 non-null  int64  
 14  Daily_Avg_Installs  90000 non-null  float64
dtypes: bool(4), float64(5), int64(4), object(2)
memory us

In [28]:
df.head()

,Category,Rating,Rating Count,Installs,Free,Price,Size_M,Content Rating,Ad Supported,In App Purchases,Editors Choice,Released_Year,Released_Month,Days_Between,Daily_Avg_Installs
0,Adventure,0.0,0.0,15,True,0.0,10.0,Everyone,False,False,False,2020,2,0,15.0
1,Productivity,0.0,0.0,58,True,0.0,3.7,Everyone,False,False,False,2019,8,10,5.8
2,Libraries & Demo,4.5,12.0,2567,True,0.0,2.5,Everyone,True,False,False,2019,9,4,641.8
3,Lifestyle,2.0,39.0,702,True,0.0,16.0,Everyone,False,False,False,2019,6,0,702.0
4,Racing,4.9,55.0,329,True,0.0,51.0,Everyone,False,False,False,2020,7,0,329.0


In [29]:
df = df.sort_values(by='Released_Year')

In [30]:
df

,Category,Rating,Rating Count,Installs,Free,Price,Size_M,Content Rating,Ad Supported,In App Purchases,Editors Choice,Released_Year,Released_Month,Days_Between,Daily_Avg_Installs
32492,Tools,3.9,494.0,99306,True,0.00,0.1,Everyone,False,False,False,2010,12,0,99306.0
35965,Finance,4.0,1269.0,317027,True,0.00,0.0,Everyone,False,False,False,2010,3,1,317027.0
34376,Books & Reference,0.0,0.0,125,False,1.99,1.3,Everyone,False,False,False,2010,8,0,125.0
57649,Productivity,3.2,17.0,8587,True,0.00,0.0,Everyone,True,False,False,2010,12,1,8587.0
31676,Tools,3.8,30.0,6454,True,0.00,0.0,Everyone,False,False,False,2010,6,0,6454.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47279,Art & Design,0.0,0.0,355,True,0.00,11.0,Everyone,True,False,False,2021,3,0,355.0
47283,Finance,0.0,0.0,663732,True,0.00,35.0,Everyone,False,False,False,2021,2,16,41483.2
31116,Social,0.0,0.0,54,True,0.00,38.0,Teen,False,False,False,2021,3,18,3.0
63430,Business,3.2,9.0,1092,True,0.00,1.7,Everyone,False,False,False,2021,2,0,1092.0


In [31]:
filepath = Path('Google-Playstore.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(filepath, index=False)